In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import IFrame, display
InteractiveShell.ast_node_interactivity = "all"

# 🔌Hybrid additive linker playground

Just a place to get linkers running.

In [1]:
from src.data import utils as du
from src.data.star import Star
from src.data.datasets import Dataset
from src.data.probabilities import Probabilities
from src.data.clusters import Clusters
from src.data.validation import Validation
from src.link.splink_linker import SplinkLinker
from src.config import link_pipeline, stopwords
from src.features.clean_complex import clean_comp_names

import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl

import uuid
from dotenv import load_dotenv, find_dotenv
import os

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

ModuleNotFoundError: No module named 'src.link.splink_linker'

## Setup

In [ ]:
star = Star(
    schema = os.getenv("SCHEMA"),
    table = os.getenv("STAR_TABLE")
)
probabilities = Probabilities(
    schema = os.getenv("SCHEMA"),
    table = os.getenv("PROBABILITIES_TABLE"),
    star = star
)
clusters = Clusters(
    schema = os.getenv("SCHEMA"),
    table = os.getenv("CLUSTERS_TABLE"),
    star = star
)

In [ ]:
# probabilities.create(overwrite=True)
# clusters.create(dim=1970, overwrite=False)

## Splink

TODO:

* Write the `linker.link` method
* Run it
* Load it into clusters
* Add `cluster_select`s to the `link_pipeline` in config

At that point I think we've got enough infra to MR the whole of this, linker, data and all. Not a pretty MR, a lot to chew, sorry reviewer.

In [ ]:
# '"hmrc"."trade__exporters"': {
#     "fact": '"hmrc"."trade__exporters"',
#     "key_fields": ["company_name", "address", "postcode"],
#     "dim": f'"{os.getenv("SCHEMA")}"."hmrc_trade__exporters__dim"',
#     "n": 3,
#     "experiment": "cm_hmrc-trade-exporters",
# }

In [ ]:
cl_x_exp = SplinkLinker(
    dataset = Dataset(
        star_id=54717,
        star=star
    ), 
    probabilities=probabilities, 
    clusters=clusters, 
    n=2
)

In [ ]:
cl_x_exp.get_data(
    cluster_select={
        '"companieshouse"."companies"': [
            "company_name",
            "postcode"
        ]
    },
    dim_select=[
        "company_name",
        "postcode"
    ]
)

In [ ]:
cl_x_exp._create_linker(
    linker_settings={
        "link_type": "link_only",
        "unique_id_column_name": "id",
        "retain_matching_columns": False,
        "retain_intermediate_calculation_columns": False,
        "blocking_rules_to_generate_predictions": [
            """
                (l.name_unusual_tokens = r.name_unusual_tokens)
                and (
                    l.name_unusual_tokens <> ''
                    and r.name_unusual_tokens <> ''
                )
            """,
            """
                (l.postcode = r.postcode)
                and (
                    l.postcode <> ''
                    and r.postcode <> ''
                )
            """,
        ],
        "comparisons": [
            cl.jaro_winkler_at_thresholds(
                "name_unusual_tokens", [0.9, 0.6], term_frequency_adjustments=True
            ),
            ctl.postcode_comparison("postcode"),
        ],
    }
)

In [ ]:
cl_x_exp._train_linker(
    train_pipeline={
        "estimate_probability_two_random_records_match": {
            "function": "estimate_probability_two_random_records_match",
            "arguments": {
                "deterministic_matching_rules": """
                    l.name_unusual_tokens = r.name_unusual_tokens
                """,
                "recall": 0.7,
            },
        },
        "estimate_u_using_random_sampling": {
            "function": "estimate_u_using_random_sampling",
            "arguments": {"max_pairs": 1e6},
        },
        "estimate_parameters_using_expectation_maximisation": {
            "function": "estimate_parameters_using_expectation_maximisation",
            "arguments": {
                "blocking_rule": """
                    l.name_unusual_tokens = r.name_unusual_tokens
                """
            },
        },
    }
)